# Dependencies

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio
import torchaudio.transforms as transforms
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from tqdm import tqdm
import json

# Check GPU availability

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load LibriSpeech dataset

In [ ]:
dataset_dir = "librispeech/dev-clean"
metadata_file = "librispeech/books.json"

# Load Metadata

In [ ]:
with open(metadata_file, "r", encoding="utf-8") as f:
    metadata = json.load(f)
print(f"Total audio samples: {len(metadata)}")

# Initialize Whisper processor

In [ ]:
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to(device)

# Define Dataset Class

In [ ]:
class LibriSpeechDataset(Dataset):
    def __init__(self, metadata, dataset_dir, processor):
        self.metadata = metadata
        self.dataset_dir = dataset_dir
        self.processor = processor
    
    def __len__(self):
        return len(self.metadata)
    
    def __getitem__(self, idx):
        sample = self.metadata[idx]
        audio_path = os.path.join(self.dataset_dir, sample["file"])
        waveform, sample_rate = torchaudio.load(audio_path)
        resampler = transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform).squeeze(0)
        input_features = self.processor.feature_extractor(waveform, sampling_rate=16000, return_tensors="pt").input_features
        return {
            "input_features": input_features.squeeze(0),
            "transcription": sample["text"]
        }


# Create Dataset

In [ ]:
dataset = LibriSpeechDataset(metadata, dataset_dir, processor)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# DataLoaders

In [ ]:
batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# Training Setup

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.7)
criterion = nn.CrossEntropyLoss()

# Training Loop

In [ ]:
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
        input_features = batch["input_features"].to(device)
        labels = processor.tokenizer(batch["transcription"], return_tensors="pt", padding=True).input_ids.to(device)
        
        optimizer.zero_grad()
        outputs = model(input_features, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    scheduler.step()
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_dataloader)}")
    torch.save(model.state_dict(), "librispeech_checkpoint.pth")


# Evaluation

In [ ]:
total_loss = 0.0
model.eval()
with torch.no_grad():
    for batch in tqdm(val_dataloader, desc="Evaluating"):
        input_features = batch["input_features"].to(device)
        labels = processor.tokenizer(batch["transcription"], return_tensors="pt", padding=True).input_ids.to(device)
        
        outputs = model(input_features, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

print(f"Validation Loss: {total_loss / len(val_dataloader):.4f}")